In [1]:
# KORA_S3S4 Data Analysis - Minimum Model

import math 
import numpy as np
import statistics
import pandas as pd
import scipy.stats
import seaborn as sns
import os
import pandas_profiling as pp
from matplotlib import pyplot as plt
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import impute
import pandas_profiling as pp
import nbimporter
import Modeling
from IPython.core.debugger import set_trace
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn import tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance
import statsmodels.api as sts
%matplotlib inline

## Loading the data and data splitting

In [2]:

KORA_Noise_noMissing = pd.read_csv('C:\\Users\\sahar.behzadi\\Desktop\\Noise2Nako\\Data\\KORA_S3_S4\\KORA_Noise_noMissing_median.csv')

In [3]:
# Inputs

X = KORA_Noise_noMissing.drop(['hyper_p', 'bp_diast', 'bp_syst'], axis = 1)
X_mini = KORA_Noise_noMissing[['sex', 'age', 'bmi', 'smoking', 'lden_org']]
X_mini.head()

# Output

Y_hyper = KORA_Noise_noMissing['hyper_p'].astype(int)
Y_SBP = KORA_Noise_noMissing['bp_syst']
Y_DBP = KORA_Noise_noMissing['bp_diast']

### Explainable AI - Categories
#### 
<img src='img/XAI_categories.png' style='width:15cm;height:8cm'>  

1. <b> Perturbation-based : </b>
Check what happens to the classifier or the regressor when some chnages happen to the input, e.g. masking the input image and see which features are more important.

- <b> Dis: </b>
slow
assumes locality (may different features are important to the task!)
perturbation may introduce artefacts (the way we manipulate the input could have some impacts on the results)


2. <b> Function-based: </b>
Find the functional perspective of the model and try to intrepret the resuts by the function. Approximate the function by e.g. Taylor decomposition (expansion)

- <b> Dis:</b>
Need to find a good root point where to perform the expansion

- <b> Adv:</b>
Can be applied to any model!

3. <b> Sampling-based:</b>
We approximate the prediction locally. (Gradient * Input)

- <b> Dis:</b>
They are very local! Do not measure global impacts.

- <b> Adv:</b>
Fast, no optimization is required

4. <b> Stuctured-based:</b>
They use the structure of the model to explain the prediction. (Layer-wise Relevance Propagation - LRP). Decompose the function by using the structure and explain the easier function and aggregate them later. 

<b>Intuition :</b> Every layer in a NN, for instance, is a composition of simpler functions (e.g., Relu) 

<img src='img/LRP.png' style='width:15cm;height:8cm'>



In [ ]:
import shap
import xgboost
shap.initjs()
model = xgboost.XGBRegressor().fit(X_mini, Y_SBP)

In [ ]:


import shap
import xgboost
shap.initjs()
model = xgboost.XGBRegressor().fit(X_mini, Y_SBP)

## SHAP Algorithm

SHAP assigns each feature an importance value for a particular prediction. 

#### 1. They introduce the perspective of viewing any explanation of a model’s prediction as a model itself, which they term the explanation model, by defining the class of additive feature attribution methods. 
For complex models, such as ensemble methods or deep networks, we cannot use the original model as its own best explanation because it is not easy to understand. Instead, we must use a simpler explanation model, which we define as any <b> interpretable approximation of the original model </b>. 
A surprising attribute of the class of additive feature attribution methods is the presence of a single unique solution in this class with three desirable properties (described below).

#### 2. Thy then show that game theory results guaranteeing a unique solution apply to the entire class of additive feature attribution methods and propose SHAP values as a unified measure of feature importance that various methods approximate.

#### 3. They propose new SHAP value estimation methods and demonstrate that they are better aligned with human intuition as measured by user studies and more effectually discriminate among model output classes than several existing methods.

In [ ]:
# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.Explainer(model)
shap_values = explainer(X_mini)

# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])
shap.plots.force(shap_values[0])

In [ ]:
shap.plots.waterfall(shap_values[1])
shap.plots.force(shap_values[1])

In [ ]:
print(X_mini.head(5))
print(shap_values[1])
shap_values[1].values / shap_values[1].data

In [ ]:
shap.plots.waterfall(shap_values[2])
shap.plots.force(shap_values[2])

In [ ]:
observations = X_mini.to_numpy()
shap.initjs() 
shap.force_plot(explainer.expected_value, explainer.shap_values(X_mini), features=observations)

In [ ]:
observations = X_mini.sample(100, random_state=42).to_numpy()
shap.initjs() 
shap.force_plot(explainer.expected_value, explainer.shap_values(observations), features=observations,
                feature_names=X_mini.columns)

In [ ]:
X_mini.head(10)

In [ ]:
shap.initjs() 
shap.summary_plot(explainer.shap_values(observations), features=observations, feature_names=X_mini.columns)

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values[:,"lden_org"], dot_size=2, x_jitter=0.5, color=shap_values[:,"age"])

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values[:,"age"], color=shap_values[:,"lden_org"])

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values[:,"bmi"], color=shap_values)

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values[:,"age"], color=shap_values)

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values[:,"sex"], dot_size=2, x_jitter=0.5, color=shap_values)

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values[:,"smoking"], dot_size=2, x_jitter=0.5, color=shap_values)

In [ ]:
shap_values

In [ ]:
X_mini.head()